In [5]:
import autogen
from dotenv import load_dotenv
import os

# Load environment variables from the .env file
load_dotenv(dotenv_path=".env")

llm_config = {"model": "gpt-4o-mini"}

# ANSI escape code for green text
GREEN = "\033[92;1m"
BLUE_BOLD = "\033[94;1m"
RESET = "\033[0m"

In [6]:
# Customer Proxy Agent
customer_proxy_agent = autogen.ConversableAgent(
    name="Customer_Proxy",
    system_message="Acts as a representative of the customer. Communicates the customer's high-level requirements, vision, and goals to the Product Owner. Provides an overarching view of the product’s purpose, expected functionalities, and outcomes. Coordinates with the Product Owner to ensure the product vision aligns with customer needs",
    code_execution_config=False,
    llm_config=llm_config,
    human_input_mode="ALWAYS",
)

# Product Owner Agent
product_owner_agent = autogen.ConversableAgent(
    name="Product_Owner",
    system_message=f"""As the Product Owner, you define and communicate the product's vision, goals, and high-level requirements to the Scrum Master. After gathering the customer's problem statement:
    - Translate requirements into high-level product goals, key performance indicators (KPIs), and essential user stories.
    - Work with the Scrum Master to establish a roadmap, prioritizing features and creating the initial product backlog.
    - Refine and communicate the prioritized backlog, including initial sprints and user stories with acceptance criteria, to the Customer Proxy and Scrum Master.
    - Oversee the initial product vision and ensure alignment with customer needs as the development plan is refined.
    """,
    code_execution_config=False,
    llm_config=llm_config,
    human_input_mode="ALWAYS",
)

# Scrum Master Agent
scrum_master_agent = autogen.ConversableAgent(
    name="Scrum_Master",
    system_message=f"""As the Scrum Master, you facilitate the Scrum process, enabling seamless collaboration. During initial planning:
    - Organize meetings between the Product Owner and Development Team to clarify project goals and requirements.
    - Communicate the product vision and high-level backlog from the Product Owner to the Project Manager, guiding initial sprint planning.
    - Collaborate with the Project Manager to outline key tasks, define roles, and structure the project timeline.
    - Prepare a preliminary backlog and sprint plan based on the Product Owner’s input, ensuring alignment with overall product objectives.
    - Deliver the initial backlog and sprint plan to the Product Owner for prioritization and refinement.""",
    code_execution_config=False,
    llm_config=llm_config,
    human_input_mode="ALWAYS",
)

# Development Team Agents

# Project Manager Agent
project_manager_agent = autogen.ConversableAgent(
    name="Project_Manager",
    system_message="""As the Project Manager, your primary responsibility is to create and coordinate the overall project plan. During the initial planning phase:
    - Receive the initial backlog and sprint requirements from the Scrum Master.
    - Coordinate with each engineer (Requirements, System, Software, Test, and Documentation) to gather user stories, effort estimates, and specific phase requirements.
    - Use historical data to inform project scope, timelines, and dependencies.
    - Compile a structured project plan, including a detailed backlog and estimated timelines, for review by the Scrum Master and Product Owner.
    - Ensure the final plan aligns with product objectives, is feasible for the development team, and meets customer expectations.
    """,
    code_execution_config=False,
    llm_config=llm_config,
    human_input_mode="ALWAYS",
)

# Requirement Engineer Agent
requirement_engineer_agent = autogen.ConversableAgent(
    name="Requirement_Engineer",
    system_message="""As the Requirement Engineer, you are responsible for defining detailed user stories and requirements. During the planning phase:
    - Convert high-level customer requirements into structured user stories, with unique identifiers and defined acceptance criteria.
    - Use past productivity rates to estimate theeffortand effort for each user story, factoring in the complexity of requirements.
    - Calculate total effort based on the volume of user stories and average productivity, ensuring alignment with overall project scope.
    - Report the detailed user stories and estimatedeffortto the Project Manager, providing a high-level summary and key requirements for the planning phase.
    calculate the {effort} to create the requirements document in hours, days, weeks, or months, follow the steps below, having cosideration a day has 8 hours:
                            - step 1. work = {the estimate of total number of requirements for the use-cases needed for the sofwtare project} 
                            - step 2. productivity rate = {estimate the complexities of the work and estimate how many requirements can be completed every day} 
                            - step 3. effort = {work}/{productivity} .
    """,
    code_execution_config=False,
    llm_config=llm_config,
    human_input_mode="ALWAYS",
)

# System Engineer Agent
system_engineer_agent = autogen.ConversableAgent(
    name="System_Engineer",
    system_message="""As the System Engineer, you design the system architecture and data model based on requirements provided. During the planning phase:
    - Analyze user stories to create a detailed system design, including architecture diagrams and a data model.
    - Document each part of the design with unique identifiers, estimatingeffortand effort using historical productivity data.
    - Calculate total effort based on the estimated pages required for design documentation.
    - Provide the Project Manager with a summary of the system design, including estimatedeffortfor completion, a list of key components, and any identified complexities.
    - Calculate the {effort} to create the design document in hours, days, weeks, or months, follow the steps below, having cosideration a day has 8 hours:
                            - step 1. work = {the estimate of total number of pages in the design document for the sofwtare project} 
                            - step 2. productivity rate = {estimate the complexities of the work and estimate how many pages can be completed every day} 
                            - step 3. effort = {work}/{productivity} .
    - Then Converts these efferts in term of User Stories. 
    """,
    code_execution_config=False,
    llm_config=llm_config,
    human_input_mode="ALWAYS",
)

# Software Engineer Agent
software_engineer_agent = autogen.ConversableAgent(
    name="Software_Engineer",
    system_message="""As the Software Engineer, you implement features and functionality based on the system design. During the planning phase:
    - Break down the system design into coding tasks, each with an estimated number of lines of code (SLOC).
    - Use past productivity data to calculateeffortrequirements for coding tasks, code reviews, and testing.
    - Provide the Project Manager with a summary of coding tasks, total estimated SLOC, andeffortrequirements for each major feature.
    - Ensure estimates includeeffortfor bug fixes, refactoring, and review meetings as needed.
     calculate the {effort} to write the source code (SLOC) in hours, days, weeks, or months, follow the steps below, having cosideration a day has 8 hours:
                            - step 1. work = {the estimate of total number of lines of code (SLOC) that will be created for the sofwtare project} 
                            - step 2. productivity rate = {estimate the complexities of the work and estimate how many SLOC can be completed every day} 
                            - step 3. effort = {work}/{productivity} .
    - Then Converts these efferts in term of User Stories. 
    """,
    code_execution_config=False,
    llm_config=llm_config,
    human_input_mode="ALWAYS",
)

# Test Engineer Agent
test_engineer_agent = autogen.ConversableAgent(
    name="Test_Engineer",
    system_message="""As the Test Engineer, you are responsible for verifying the quality and functionality of the product. During the planning phase:
    - Develop a test plan with user stories defining individual test cases, including acceptance criteria from the Product Owner.
    - Estimateeffortfor test case creation, execution, defect tracking, and retesting based on past productivity data.
    - Report to the Project Manager with a structured test plan, estimated number of test cases, and overalleffortrequired for the testing phase.
    - Summarize key testing tasks, including preparation, defect resolution, and documentation of results.
    calculate the {effort} to write and execute the test cases in the test plan in hours, days, weeks, or months, follow the steps below, having cosideration a day has 8 hours:
                            - step 1. work = {the estimate of total number of test cases that will be created adn executed for the sofwtare project} 
                            - step 2. productivity rate = {estimate the complexities of the work and estimate how many test cases can be completed every day} 
                            - step 3. effort = {work}/{productivity} 
    - Then Converts these efferts in term of User Stories. 
    """,
    code_execution_config=False,
    llm_config=llm_config,
    human_input_mode="ALWAYS",
)

# Documentation Engineer Agent
documentation_engineer_agent = autogen.ConversableAgent(
    name="Documentation_Engineer",
    system_message="""As the Documentation Engineer, you create comprehensive user and training documentation. During the planning phase:
    - Outline documentation tasks, including user guides and training materials, based on user stories and completed product features.
    - Estimate theeffortrequired for each documentation task based on historical productivity data, includingeffortfor review and feedback.
    - Provide the Project Manager with a summary report of documentation tasks, estimated total pages, andeffortrequirements.
    - Highlight any dependencies on development or testing and plan for incremental updates as features are completed.
    calculate the {effort} to write documentation pages in the documentation adn traingin phase in hours, days, weeks, or months, follow the steps below, having cosideration a day has 8 hours:
                            - step 1. work = {the estimate of total number of documentation pages that will be created adn executed for the sofwtare project} 
                            - step 2. productivity rate = {estimate the complexities of the work and estimate how many pages can be completed every day} 
                            - step 3. effort = {work}/{productivity} 
    - Then Converts these efferts in term of User Stories. 
    """,
    code_execution_config=False,
    llm_config=llm_config,
    human_input_mode="ALWAYS",
)

[autogen.oai.client: 11-07 17:34:33] {164} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.
[autogen.oai.client: 11-07 17:34:33] {164} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.
[autogen.oai.client: 11-07 17:34:34] {164} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.
[autogen.oai.client: 11-07 17:34:34] {164} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.
[autogen.oai.client: 11-07 17:34:35] {164} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.
[autogen.oai.client: 11-07 17:34:35] {164} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.
[autogen.oai.client: 11-07 17:34:35] {164} WARNING - The API key specified is not a valid OpenAI format; i

In [7]:
# Initial conversation: Customer Proxy to Product Owner
customer_message = (
    "I want to create an AI-powered computer vision system that detects anomalies in the skin, such as cancer and psoriasis. "
    "The system should process high-resolution images, provide real-time diagnostics, and support doctors with detailed reports. "
    "This project should prioritize accuracy, speed, and usability for healthcare providers."
    "Please document the initial product vision, objectives, and KPIs, including primary user stories that will guide development and outline a high-level product backlog focused on prioritized features and metrics to assess product value. "
)

# Product Owner’s prompt to Scrum Master
product_owner_to_scrum_master_prompt = (
    "I have gathered the requirements from the Customer Proxy for an AI-powered skin anomaly detection system, "
    "which includes high-resolution image processing, real-time diagnostics, and detailed reporting capabilities. "
    "Please initiate the sprint planning with a focus on prioritizing critical milestones based on the system's primary needs: core detection accuracy, processing speed, and usability. "
    "We estimate the project could be completed within approximately 3 to 5 sprints, but please adjust as needed to best align with the team's velocity and project goals. "
    "For the initial sprints, consider focusing on developing the core detection algorithm and establishing the image processing pipeline, followed by diagnostics and reporting features. "
    "Coordinate this sprint planning with the development team, ensuring each sprint has a structured backlog and clear effort estimates to guide development effectively."
)

# Scrum Master’s prompt to Project Manager
scrum_master_to_project_manager_prompt = (
    "The Product Owner has shared the project vision and initial backlog for the skin anomaly detection system, including goals for high-resolution image processing, real-time diagnostics, and reporting. "
    "Based on this, I have drafted an initial sprint plan covering key roles, estimated timelines, and primary milestones. "
    "As the Project Manager, please develop a comprehensive project plan that breaks down detailed user stories, prioritizes them, and includes specific effort estimates for each task. "
    "Ensure that each user story in the backlog is clearly defined with structured tasks and relevant effort estimates to guide the development team. "
    "Coordinate with each engineer (e.g., Requirements Engineer, System Engineer, Software Engineer, and Test Engineer) to gather specific use cases and effort details, ensuring that the backlog is both complete and actionable for the upcoming sprints."
)

# Project Manager’s prompt to Requirement Engineer
project_manager_to_requirement_engineer_prompt = (
    "The Scrum Master has shared the initial sprint planning requirements based on the Product Owner’s input. "
    "As the Requirements Engineer, your main task is to create detailed user stories for requirements gathering and documentation, including effort estimates. "
    "Please provide a structured report summarizing the user stories (a summary for each story, but the full list should be shared with me), along with effort estimates and calculations for the requirement engineering phase to support sprint planning."
    
    "At the end of the report, please summarize the following: total number of user stories, total number of requirements, and the overall effort for the requirement engineering phase."

    "To aid your estimates, you may reference historical productivity data from previous projects. "
    "**Since this data is based on a waterfall model, it is crucial to adapt it to align with the agile development model for accurate estimates.** "
    "Please ensure all effort estimates reflect an agile approach, incorporating iterations and adjustments typical in agile processes."

    "Here is the amount of work and productivity rate for the requirement engineering phase for the previous projects:"

        "Project 1"
        "Task                                    Amount of Work      Productivity Rate"
        "Write Requirement                           112 Req                 5 Req/Hour"
        "Review Requirments                                                            "
        "       Preparation for Review                112  Req               18 Req/Hour"
        "       Review Meeting                        112 Req                28 Req/Hour"
        "Rework                                       136 defects            10 defects/Hour"

        "Project 2"
        "Task                                    Amount of Work      Productivity Rate"
        "Write Requirement                           389 Req                 3 Req/Hour"
        "Review Requirments                                                      "
        "       Preparation for Review                389  Req               5 Req/Hour"
        "       Review Meeting                        389 Req                8 Req/Hour"
        "Rework                                       413 defects            5 defects/Hour"

        "Project 3"
        "Task                                    Amount of Work      Productivity Rate"
        "Write Requirement                           210 Req                 4 Req/Hour"
        "Review Requirments                                                      "
        "       Preparation for Review                210  Req               12 Req/Hour"
        "       Review Meeting                        210 Req                18 Req/Hour"
        "Rework                                       239 defects            8 defects/Hour"
    
    "Once completed, please report back to me so we can proceed with the next steps in the project plan."
)

# Project Manager’s prompt to System Engineer
project_manager_to_system_engineer_prompt = (
    "The Requirements Engineer has completed the initial set of user stories. Based on these, please draft user stories for system design "
    "and data model creation, including detailed effort estimates for each task. Use historical data as a reference to ensure accuracy, and provide a few example user stories in your report. "
    
    "Please structure your report as follows: a summary list of user stories, detailed effort estimates, and calculations for the system engineering phase to support sprint planning. "
    "At the end, summarize the following metrics: total number of user stories, total number of requirements, and the total effort estimated for this phase."

    "To aid your estimates, you may reference historical productivity data from previous projects. "
    "**Since this data is based on a waterfall model, it is crucial to adapt it to align with the agile development model for accurate estimates.** "
    "Please ensure all effort estimates reflect an agile approach, incorporating iterations and adjustments typical in agile processes."
    
     "Here is the amount of work and productivity rate for the system engineering phase for the previous projects:"

        "Project 1"
        "Task                                    Amount of Work      Productivity Rate"
        "Write Analysis Document                      66 Pages                 5 Pages/Hour"
        "Review AD                                                            "
        "       Preparation for AD                 66 Pages                    4 Pages/Hour"
        "       Review Meeting                        66 Pages                9 Pages/Hour"
        "Rework                                       88 defects            6 defects/Hour"
        "Write Design Document                       123 defects                 6 Page/Hour"
        "Review DD                                                            "
        "       Preparation for DD                 89 Pages              4 Req/Hour"
        "       Review Meeting                        89 Pages                8 Pages/Hour"
        "Rework                                       188 defects            5 defects/Hour"
        "Write Data Model (DM)                        28 Pages               1page/4hours"
        "Review DM                                                                     "
        "       Preparation for DM                 28 Pages                   4 page/hour"
        "       Review Meeting                        28 Pages                5 page/hour"
        "Rework                                       88 defects            6 defects/Hour"

        "Project 2"
        "Task                                                          Amount of Work      Productivity Rate"
        "Write Analysis/Design Document                                     234 Pages                 4 Pages/Hour"
        "Review Analysis/Design Document                                                           "
        "       Preparation for Analysis/Design Document                   234 Pages              5 Pages/Hour"
        "       Review Meeting                                             234 Pages               8 Pages/Hour"
        "Rework                                                             509 defects            7 defects/Hour"
        "Write Data Model (DM)                                             62 Pages               1 page/ hour"
        "Review DM                                                                     "
        "       Preparation for DM                    62 Pages                 4 page/hour"
        "       Review Meeting                        62 Pages                 10 page/hour"
        "Rework                                       378 defects            6 defects/Hour"

        "Project 3"
        "Task                                    Amount of Work      Productivity Rate"
        "Write Analysis Document                      172 Pages                 5 Pages/Hour"
        "Review AD                                                            "
        "       Preparation for AD                   172 Pages                    5 Pages/Hour"
        "       Review Meeting                        172 Pages                9 Pages/Hour"
        "Rework                                       347 defects            7 defects/Hour"
        "Write Design Document                       220 Pages                 5 Pages/Hour"
        "Review DD                                                            "
        "       Preparation for DD                    220 Pages              5 Pages/Hour"
        "       Review Meeting                        220 Pages               8 Pages/Hour"
        "Rework                                       472 defects            6 defects/Hour"
    
    "Once you have completed the report, please send it back to me so we can move forward with the next steps in the project plan."
)

# Project Manager’s prompt to Software Engineer
project_manager_to_software_engineer_prompt = (
    "The System Engineer has completed the design phase user stories. As the Software Engineer, your task is to create user stories for coding tasks, including effort estimates for each. "
    "Please structure your report with a summary of all user stories (provide a brief summary list, but ensure the full list of user stories is accessible to me), detailed effort estimates, and calculations for the software engineering phase to support sprint planning."
    
    "At the end of your report, please summarize the following metrics: the total number of user stories, the total number of requirements, and the overall effort for the software engineering phase."

    "To aid your estimates, you may reference historical productivity data from previous projects. "
    "**Since this data is based on a waterfall model, it is crucial to adapt it to align with the agile development model for accurate estimates.** "
    "Please ensure all effort estimates reflect an agile approach, incorporating iterations and adjustments typical in agile processes."
    
    "Here is the amount of work and productivity rate for the software engineering phase for the previous projects:"

        "Project 1"
        "Task                                    Amount of Work      Productivity Rate"
        "Write Code                                  3800 SLOC                 5 SLOC/Hour"
        "Unit Testing                                                                     "
        "       Prepare/Execute Test Cases            256 test cases           10 Test Cases/Day"
        "       Fix Found Defects                     179 Defects              8 Defects/Day"
        "       Test Fixed Defects                    179 Defects              12 Defects/Day"
        "Code Inspection                                                                   "
        "       Preparation for Code Inspection       3800 SLOC                 113 SLOC/Hour"
        "       Code Inspection Meeting               3800 SLOC                 189 SLOC/Hour"
        "Rework                                       188 defects                5 defects/Hour"

        "Project 2"
        "Task                                    Amount of Work           Productivity Rate"
        "Write Code                                  9725 SLOC                 5 SLOC/Hour"
        "Unit Testing                                                                     "
        "       Prepare/Execute Test Cases            1045 test cases           5 Test Cases/Day"
        "       Fix Found Defects                     902 Defects              12 Defects/Day"
        "       Test Fixed Defects                    902 Defects              8 Defects/Day"
        "Code Inspection                                                                   "
        "       Preparation for Code Inspection       9725 SLOC                 90 SLOC/Hour"
        "       Code Inspection Meeting               9725 SLOC                 135 SLOC/Hour"
        "Rework                                       1230 defects                5 defects/Hour"

        "Project 3"
        "Task                                    Amount of Work           Productivity Rate"
        "Write Code                                  3800 SLOC                 5 SLOC/Hour"
        "Unit Testing                                                                     "
        "       Prepare/Execute Test Cases            332 test cases           25 Test Cases/Day"
        "       Fix Found Defects                     266 Defects              10 Defects/Day"
        "       Test Fixed Defects                    266 Defects              10 Defects/Day"
        "Code Inspection                                                                   "
        "       Preparation for Code Inspection       3800 SLOC                 102 SLOC/Hour"
        "       Code Inspection Meeting               3800 SLOC                 102 SLOC/Hour"
        "Rework                                       334 defects                5 defects/Hour"
    
    "Once completed, report back to me so I can proceed with the next steps in the project plan."
)

# Project Manager’s prompt to Test Engineer
project_manager_to_test_engineer_prompt = (
    "The Software Engineer has completed the coding tasks user stories. Based on these, please create user stories for test planning and execution, including effort estimates for writing test cases, execution, and rework. "
    "Provide a structured report with the following: a summary list of user stories (full details of user stories should be accessible to me), detailed effort estimates, and calculations for the test engineering phase to support sprint planning."
    
    "At the end of your report, please summarize the following: total number of user stories, total number of requirements, and overall effort for the test engineering phase."


    "For your estimates, please reference historical productivity data from previous projects."
    "**Since this data is based on a waterfall model, it is crucial to adapt it to align with the agile development model for accurate estimates.**"
    " Please ensure all effort estimates reflect an agile approach, incorporating iterations and adjustments typical in agile processes."
    
    "Here is the amount of work and productivity rate for the test engineering phase for the previous projects:"

        "Project 1"
        "Task                                    Amount of Work      Productivity Rate"
        "Write Test Plan (TP)                    166 pages            5 Pages/Day"
        "Review TP                                                            "
        "       Preparation for Review            166 Pages           4 Pages/Hour"
        "       Review Meeting                    166 Pages            8 Pages/Hour"
        "Rework                                   255 defects          5 defects/Hour"
        "Execute Test Plan                        215 Test Cases     9Test Cases/day"
        "Fix Found Defects                        213 Defects          14 Defects/day"
        "Test Fixed Defects                       213 Defects          17 Defects/day"

        "Project 2"
        "Task                                    Amount of Work      Productivity Rate"
        "Write Test Plan (TP)                    417 pages            8 Pages/Day"
        "Review TP                                                            "
        "       Preparation for Review            417 Pages           5 Pages/Hour"
        "       Review Meeting                    417 Pages            10 Pages/Hour"
        "Rework                                   502 defects          5 defects/Hour"
        "Execute Test Plan                        1167 Test Cases      8 Test Cases/day"
        "Fix Found Defects                        645 Defects          5 Defects/day"

        "Project 3"
        "Task                                    Amount of Work      Productivity Rate"
        "Write Test Plan (TP)                    205 pages            8 Pages/Day"
        "Review TP                                                            "
        "       Preparation for Review            205 Pages           5 Pages/Hour"
        "       Review Meeting                    205 Pages            10 Pages/Hour"
        "Rework                                   101 defects          4 defects/Hour"
        "Execute Test Plan                        194 Test Cases      8 Test Cases/day"
        "Fix Found Defects                        113 Defects          5 Defects/day"
        "Test Fixed Defect                        113 Defects          10 Defects/day"
    
    "Once completed, report back to me so I can proceed with the next steps in the project plan."
)

# Project Manager’s prompt to Documentation Engineer
project_manager_to_documentation_engineer_prompt = (
    "The Test Engineer has completed user stories for testing. As the Documentation Engineer, please create user stories for documentation tasks, including effort estimates for each task. "
    "Provide a structured report with the following: a summary list of user stories (ensure the full details of user stories are accessible to me), detailed effort estimates, and calculations for the documentation phase to support sprint planning."
    
    "At the end of your report, please summarize the following: total number of user stories, total number of requirements, and the overall effort for the documentation engineering phase."

    "The provided historical data for the documentation engineering phase is for the waterfall model, and the productivity rate is based on the number of requirements per hour. But you need to relavant this data to the agile model. "
    "You may use the following information to estimate the Amount of work, Productivity Rate and efforts for the documentation engineering phase:"
    
    "Here is the amount of work and productivity rate for the documentation engineering phase for the previous projects:"

    "Project 1"
    "Task                                    Amount of Work      Productivity Rate"
    "User Documentation                     134 Pages                 4 Pages/Hour"
    "Review User Documentation                                                    "
    "       Preparation for Review            134 Pages               5 Pages/Hour"
    "       Review Meeting                    134 Pages                7 Pages/Hour"
    "Rework                                   174 defects            6 defects/Hour"

    "Project 2"
    "Task                                    Amount of Work      Productivity Rate"
    "User Documentation                      389 Pages                 6 Pages/Hour"
    "Review User Documentation                                                    "
    "       Preparation for Review            389 Pages                5 Pages/Hour"
    "       Review Meeting                    389 Pages                7 Pages/Hour"
    "Rework                                   532 defects            4 defects/Hour"

    "Project 3"
    "Task                                    Amount of Work      Productivity Rate"
    "User Documentation                      177 Pages                 5 Pages/Hour"
    "Review User Documentation                                                    "
    "       Preparation for Review            177 Pages                5 Pages/Hour"
    "       Review Meeting                    177 Pages                7 Pages/Hour"
    "Rework                                   236 defects            5 defects/Hour"

    "Once completed, report back to me so I can proceed with the next steps in the project plan."
)

project_manager_compiles_backlog = (
    "The outputs from all engineers have been received. I have compiled a complete backlog with user stories, time, and work estimates for each phase. "
    "This backlog is now ready for review and structuring for the initial Scrum planning. "
    "Please find the backlog summary with user stories, effort estimates, and phase-specific estimates in the attached report. "
    
    "When generating the final backlog, organize user stories into sprints, focusing each sprint on a specific project milestone or functionality. Ensure each sprint has a clear focus that aligns with key project objectives, such as core detection, high-resolution processing, or diagnostics optimization. "
    
    "Estimate an approximate number of sprints required to complete the project and assign user stories to each sprint based on their priority and relevance to the sprint's focus. Each sprint should have defined objectives that contribute to the gradual achievement of the project’s goals. "
    
    "Once the plan is finalized, report back with a summary that includes the total number of sprints, total number of user stories, the primary focus of each sprint, and the total effort estimated for the project."
)

scrum_master_structures_plan = (
    "The Project Manager has compiled a complete backlog, which I have reviewed and structured for the initial Scrum plan. "
    "Based on the total user stories and effort estimates, organize the tasks into a series of sprints with each sprint focusing on a specific milestone or functionality. "
    "Prioritize user stories according to their relevance to the sprint’s focus, such as implementing core detection, optimizing image processing, or refining diagnostic features, ensuring each sprint directly supports the project’s overall objectives. "
    
    "Define a clear objective for each sprint that aligns with product goals and plan for gradual project progress through these targeted milestones. Adjust the order of tasks if needed to maintain a balanced workload and optimize project flow. "
    
    "Once the plan is finalized, report back with a summary that includes the total number of sprints, total number of user stories, the primary focus and objectives for each sprint, and the total effort estimated for the project."
)

product_owner_finalize_backlog = (
    "The Scrum Master has provided an initial Scrum plan. I have reviewed and prioritized the user stories in the backlog, "
    "making adjustments to ensure each sprint aligns with our product goals and builds toward key project milestones. "
    
    "The final sprint breakdown includes specific objectives and prioritized user stories for each sprint, with each phase focusing on a core functionality "
    "or improvement area, such as core detection, image processing optimization, or diagnostic enhancements. "
    
    "This report provides a detailed summary of the sprint goals, the total number of sprints, key user stories within each sprint, and an estimated timeline for project completion. "
    "Each sprint’s focus is outlined to ensure alignment with overarching product goals and to offer a clear roadmap of prioritized features and expected outcomes for review by the Customer Proxy."
)

product_owner_to_customer_proxy = (
   "We have completed the initial planning phase for the project. Based on your requirements, we have structured the project "
    "into prioritized sprints with defined user stories and estimated effort for each phase. "
    
    "This project plan organizes user stories by priority to ensure key functionalities are addressed early on, with each sprint focused on specific objectives "
    "that gradually build towards the final product. "
    
    "The overall project plan is now prepared for your review, including the total number of sprints, a summary of each sprint's primary goals, "
    "and an estimated completion timeline. We look forward to moving forward with the first sprint."
)


In [8]:
# Store engineer responses
engineer_outputs = {}

def print_next_speaker(i, queue):
    if i + 1 < len(queue):
        next_speaker = queue[i + 1]['recipient'].name.replace("_", " ")
        print(f"{GREEN}Next Speaker: {next_speaker}{RESET}")
    else:
        print(f"{GREEN}Final Step: Project Manager compiles the final report and sends it to the Scrum Master{RESET}")

def extract_engineer_work(chat_result):
    if chat_result and chat_result.chat_history:
        # Assuming the last entry in chat_history is the engineer's response (output)
        return chat_result.chat_history[-1]['content']
    return 'No response available from the engineer'

def compile_final_project_plan(engineer_outputs):
    final_project_plan = "### Final Project Plan\n\n"
    final_project_plan += "The Initiation and Planning phases are complete. Below is the detailed project plan outlining work completed by each agent:\n\n"
    
    chat_log_index = 1
    
    # Aggregate each agent's structured output
    for agent, details in engineer_outputs.items():
        final_project_plan += (
            f"{BLUE_BOLD}#### Chat Log {chat_log_index}: {agent.replace('_', ' ')} Phase{RESET}\n"
            f"- **Speaker:** {details['speaker'].replace('_', ' ')}\n"
            f"- **Recipient:** {details['recipient'].replace('_', ' ')}\n"
            f"- **Message:**\n{details['input_message']}\n\n"
            f"- **Response:**\n{details['response']}\n\n"
        )
        chat_log_index += 1  # Increment for each phase

    final_project_plan += "### Conclusion:\n"
    final_project_plan += (
        "The project has completed initial stages, including backlog creation with initial user stories estimates. "
        "The project is now ready for further estimating the number of sprints required based on the generated backlog and divide the backlog items for each sprint."
    )
    
    return final_project_plan

def initiate_nested_chats(nested_chat_queue):
    accumulated_work = ""  # Track responses

    # Process each conversation in nested queue
    for i, chat in enumerate(nested_chat_queue):
        sender = chat['sender']
        recipient = chat['recipient']
        message = chat['message']

        # Append accumulated work for next agent
        if accumulated_work:
            message += f"\n\nHere is the combined work from previous agents:\n{accumulated_work}"

        print_next_speaker(i, nested_chat_queue)  # Print next speaker

        # Initiate chat, capture response
        output = sender.initiate_chat(
            agent=sender,
            recipient=recipient,
            message=message
        )

        # Extract response and store in engineer_outputs
        response = extract_engineer_work(output)
        engineer_outputs[recipient.name] = {
            "speaker": sender.name,
            "recipient": recipient.name,
            "input_message": message,
            "response": response
        }

        # Append recipient's response to accumulated work
        accumulated_work += f"\n\nWork from {recipient.name.replace('_', ' ')}:\n{response}"

    # Compile the final project plan from all responses
    final_message_to_team = compile_final_project_plan(engineer_outputs)

    # Sequentially send the final project plan up the chain
    send_final_report(final_message_to_team)

def send_final_report(final_message_to_team):
    # Chain of communication for final project plan
    print(f"{GREEN}Project Manager sends the compiled backlog to Scrum Master{RESET}")
    scrum_master_response = project_manager_agent.initiate_chat(
        agent=project_manager_agent,
        recipient=scrum_master_agent,
        message=project_manager_compiles_backlog + "\n\n" + final_message_to_team
    )

    print(f"{GREEN}Scrum Master structures the backlog for sprint planning and sends it to Product Owner{RESET}")
    product_owner_response = scrum_master_agent.initiate_chat(
        agent=scrum_master_agent,
        recipient=product_owner_agent,
        message=scrum_master_structures_plan + "\n\n" + extract_engineer_work(scrum_master_response)
    )

    print(f"{GREEN}Product Owner finalizes the sprint plan and sends it to Customer Proxy{RESET}")
    product_owner_response = product_owner_agent.initiate_chat(
        agent=product_owner_agent,
        recipient=customer_proxy_agent,
        message=product_owner_finalize_backlog + "\n\n" + extract_engineer_work(product_owner_response)
    )

    print(f"{GREEN}Product Owner provides the final overview to Customer Proxy{RESET}")
    product_owner_agent.initiate_chat(
        agent=product_owner_agent,
        recipient=customer_proxy_agent,
        message=product_owner_to_customer_proxy + extract_engineer_work(product_owner_response)
    )

# Define the nested chat queue with sender and recipient for each conversation step
nested_chat_queue = [
    {"sender": customer_proxy_agent, "recipient": product_owner_agent, "message": customer_message},
    {"sender": product_owner_agent, "recipient": scrum_master_agent, "message": product_owner_to_scrum_master_prompt},
    {"sender": scrum_master_agent, "recipient": project_manager_agent, "message": scrum_master_to_project_manager_prompt},
    {"sender": project_manager_agent, "recipient": requirement_engineer_agent, "message": project_manager_to_requirement_engineer_prompt},
    {"sender": project_manager_agent, "recipient": system_engineer_agent, "message": project_manager_to_system_engineer_prompt},
    {"sender": project_manager_agent, "recipient": software_engineer_agent, "message": project_manager_to_software_engineer_prompt},
    {"sender": project_manager_agent, "recipient": test_engineer_agent, "message": project_manager_to_test_engineer_prompt},
    {"sender": project_manager_agent, "recipient": documentation_engineer_agent, "message": project_manager_to_documentation_engineer_prompt},
]

# Start the adjusted nested chat sequence
initiate_nested_chats(nested_chat_queue=nested_chat_queue)


Next Speaker: Scrum Master
Customer_Proxy (to Product_Owner):

I want to create an AI-powered computer vision system that detects anomalies in the skin, such as cancer and psoriasis. The system should process high-resolution images, provide real-time diagnostics, and support doctors with detailed reports. This project should prioritize accuracy, speed, and usability for healthcare providers.Please document the initial product vision, objectives, and KPIs, including primary user stories that will guide development and outline a high-level product backlog focused on prioritized features and metrics to assess product value. 

--------------------------------------------------------------------------------

>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> USING AUTO REPLY...
Product_Owner (to Customer_Proxy):

### Initial Product Vision
To develop an AI-powered computer vision system that empowers healthcare providers to detect skin anomalies—including cancer and psoriasis—efficiently and accur